In [23]:
#!pip install simple_salesforce
#!pip install pymysql
#!pip install sqlalchemy

In [159]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [160]:
# Make sure to use your own `config.py` file. Consider ensuring that these variable names are in sync
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [161]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [162]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Prepare ETL for the Course data

In [163]:
course_data_df = pd.read_sql("SELECT * FROM course", conn)
course_data_df.head(30)

,ID_Course,CourseCode,CourseName,CreditHours,BootCampCourse
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0
5,6,CIS-438,Database Administration,5,0
6,7,CIS-430,Business Systems Programming I,5,0
7,8,CIS-435,Business Systems Programming II,5,0


In [164]:
course_data_df.rename(columns={
    'CourseCode':'Course_Code__c',
    'CourseName':'Course_Name__c',
    'CreditHours':'Credit_Hours__c',
    'BootCampCourse':'Boot_Camp_Course__c',    
}, inplace=True)
course_data_df.head()

,ID_Course,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [165]:
course_data_df = course_data_df[['Course_Code__c', 'Course_Name__c', 'Credit_Hours__c', 'Boot_Camp_Course__c']]
course_data_df.head()

,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,BC-WEBDEV,Full Stack Web Development,12,1
2,BC-UIUX,User Interface/User Experience,12,1
3,CIS-349,Introduction to Databases,5,0
4,CIS-405,Database Programming,5,0


In [166]:
course_data_records = course_data_df.to_dict('records')
course_data_records

[{'Course_Code__c': 'BC-DATAVIZ',
  'Course_Name__c': 'Data Visualization and Analytics',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'BC-WEBDEV',
  'Course_Name__c': 'Full Stack Web Development',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'BC-UIUX',
  'Course_Name__c': 'User Interface/User Experience',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'CIS-349',
  'Course_Name__c': 'Introduction to Databases',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-405',
  'Course_Name__c': 'Database Programming',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-438',
  'Course_Name__c': 'Database Administration',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-430',
  'Course_Name__c': 'Business Systems Programming I',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-435',
  'Course_Name__c':

In [167]:
for rec in course_data_records:

    record = {
        'Course_Code__c': rec['Course_Code__c'],
        'Course_Name__c': rec['Course_Name__c'],
        'Credit_Hours__c': rec['Credit_Hours__c'],
        'Boot_Camp_Course__c': rec['Boot_Camp_Course__c'],
    }
    
    try:
        sf.Course__c.create(record)
    except Exception as e:
        print(e)

In [168]:
# Bulk 
#sf.bulk.Course__c.insert(course_data_records)

## Create Course Lookup Table
You will use this later to crosswalk the course code with the primary key from the `Course` table

It is important to note that we will be querying **Salesforce** to retrieve the record IDs 

In [169]:
course_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT Course_Code__c, Name FROM Course__c")
for row in data:
    rec = {
        'ID_Course__c': row['Name'], # this is a critical line of code
        'Course_Code__c': row['Course_Code__c']
    }
    course_lookup_list.append(rec)
    
course_lookup_list

[{'ID_Course__c': 'a004x000003VU7c', 'Course_Code__c': 'BC-UIUX'},
 {'ID_Course__c': 'a004x000003VU7r', 'Course_Code__c': 'CIS-438'},
 {'ID_Course__c': 'a004x000003VU7m', 'Course_Code__c': 'CIS-405'},
 {'ID_Course__c': 'a004x000003VU7h', 'Course_Code__c': 'CIS-349'},
 {'ID_Course__c': 'a004x000003VU7S', 'Course_Code__c': 'BC-DATAVIZ'},
 {'ID_Course__c': 'a004x000003VU7w', 'Course_Code__c': 'CIS-430'},
 {'ID_Course__c': 'a004x000003VU81', 'Course_Code__c': 'CIS-435'},
 {'ID_Course__c': 'a004x000003VU7X', 'Course_Code__c': 'BC-WEBDEV'}]

In [170]:
course_lookup_df = pd.DataFrame(course_lookup_list)
course_lookup_df

,ID_Course__c,Course_Code__c
0,a004x000003VU7c,BC-UIUX
1,a004x000003VU7r,CIS-438
2,a004x000003VU7m,CIS-405
3,a004x000003VU7h,CIS-349
4,a004x000003VU7S,BC-DATAVIZ
5,a004x000003VU7w,CIS-430
6,a004x000003VU81,CIS-435
7,a004x000003VU7X,BC-WEBDEV


In [171]:
# Query the `Class` table from MySQL
query = '''
    SELECT 
        co.CourseCode,
        cl.*
    FROM 
        class cl
        INNER JOIN course co
        ON cl.ID_Course = co.ID_Course

'''

class_data_df = pd.read_sql(query, conn)
class_data_df.head()

,CourseCode,ID_Class,ID_Course,Section,StartDate,EndDate
0,BC-DATAVIZ,1,1,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14
1,BC-DATAVIZ,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,BC-WEBDEV,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,BC-UIUX,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [172]:
class_data_df.rename(columns={
    'CourseCode':'Course_Code__c',
    'Section':'Section__c',
    'StartDate':'Start_Date__c',
    'EndDate':'End_Date__c',
    #'CourseCode':'Course_Code__c'
}, inplace=True)


class_data_df = class_data_df[['Course_Code__c', 'Section__c', 'Start_Date__c', 'End_Date__c']]
class_data_df

,Course_Code__c,Section__c,Start_Date__c,End_Date__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14
1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08
2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07
3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07


## Join the Class DataFrame with the Course lookup table
This join is necessary to successfully lookup the foreign key for the Course table 

In [173]:
class_data_df = pd.merge(class_data_df, course_lookup_df, how='left')
#class_data_df.drop(columns = ['ID_Class','ID_Course','CourseName','CreditHours','BootCampCourse','Course_Code__c'], inplace=True)

class_data_df.head()

,Course_Code__c,Section__c,Start_Date__c,End_Date__c,ID_Course__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14,a004x000003VU7S
1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08,a004x000003VU7S
2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07,a004x000003VU7X
3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07,a004x000003VU7c


In [174]:
class_data_df
class_data_df['Start_Date__c'] = pd.to_datetime(class_data_df['Start_Date__c']).dt.date
class_data_df['End_Date__c'] = pd.to_datetime(class_data_df['End_Date__c']).dt.date

class_data_df.head()


,Course_Code__c,Section__c,Start_Date__c,End_Date__c,ID_Course__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14,a004x000003VU7S
1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08,a004x000003VU7S
2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07,a004x000003VU7X
3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07,a004x000003VU7c


In [175]:
class_data_records = class_data_df.to_dict(orient='records')
class_data_records

[{'Course_Code__c': 'BC-DATAVIZ',
  'Section__c': 'GWU-ARL-DATA-PT-09-0',
  'Start_Date__c': datetime.date(2020, 9, 16),
  'End_Date__c': datetime.date(2020, 3, 14),
  'ID_Course__c': 'a004x000003VU7S'},
 {'Course_Code__c': 'BC-DATAVIZ',
  'Section__c': 'GWDC201805DATA3',
  'Start_Date__c': datetime.date(2018, 5, 15),
  'End_Date__c': datetime.date(2018, 11, 8),
  'ID_Course__c': 'a004x000003VU7S'},
 {'Course_Code__c': 'BC-WEBDEV',
  'Section__c': 'GWARL201905WEB3',
  'Start_Date__c': datetime.date(2019, 5, 14),
  'End_Date__c': datetime.date(2019, 11, 7),
  'ID_Course__c': 'a004x000003VU7X'},
 {'Course_Code__c': 'BC-UIUX',
  'Section__c': 'GWARL201905UIUX3',
  'Start_Date__c': datetime.date(2019, 5, 14),
  'End_Date__c': datetime.date(2019, 11, 7),
  'ID_Course__c': 'a004x000003VU7c'}]

## Insert `Class` Records into Salesforce

In [176]:
for rec in class_data_records:
 
    record = {
        'ID_Course__c': rec['ID_Course__c'],
        'Section__c': rec['Section__c'],
        'Start_Date__c': str(rec['Start_Date__c']),
        'End_Date__c': str(rec['End_Date__c']),
    }
    
    try:
        sf.Class__C.create(record)
    except Exception as e:
        print(e)

## Prepare ETL for te Staff Data

In [177]:
staff_data_df = pd.read_sql("SELECT * FROM staff", conn)
staff_data_df.head()

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Luongo,Darick,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Rodney,Heather,Nicole,1994-06-07


In [178]:
staff_data_df.rename(columns={
    'EmployeeID':'EmployeeID__c',
    'LastName': 'Last_Name__c',
    'FirstName': 'First_Name__c',
    'MiddleName': 'Middle_Name__c',
    'BirthDate': 'Birth_Date__c',
}, inplace=True)
staff_data_df.head()

,ID_Staff,EmployeeID__c,Last_Name__c,First_Name__c,Middle_Name__c,Birth_Date__c
0,1,000184220,Luongo,Darick,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Rodney,Heather,Nicole,1994-06-07


In [179]:
staff_data_df = staff_data_df[['EmployeeID__c', 'Last_Name__c', 'First_Name__c', 'Middle_Name__c','Birth_Date__c']]
staff_data_df.head()

,EmployeeID__c,Last_Name__c,First_Name__c,Middle_Name__c,Birth_Date__c
0,000184220,Luongo,Darick,Nico,1995-07-05
1,000130109,Sanford,Gemini,Blair,1992-04-22
2,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,000159108,Rodney,Heather,Nicole,1994-06-07


In [180]:
staff_data_records = staff_data_df.to_dict('records')
staff_data_records

[{'EmployeeID__c': '000184220',
  'Last_Name__c': 'Luongo',
  'First_Name__c': 'Darick',
  'Middle_Name__c': 'Nico',
  'Birth_Date__c': datetime.date(1995, 7, 5)},
 {'EmployeeID__c': '000130109',
  'Last_Name__c': 'Sanford',
  'First_Name__c': 'Gemini',
  'Middle_Name__c': 'Blair',
  'Birth_Date__c': datetime.date(1992, 4, 22)},
 {'EmployeeID__c': '000160655',
  'Last_Name__c': 'Williams',
  'First_Name__c': 'Dartanion',
  'Middle_Name__c': 'De Angelo',
  'Birth_Date__c': datetime.date(1993, 5, 21)},
 {'EmployeeID__c': '000159108',
  'Last_Name__c': 'Rodney',
  'First_Name__c': 'Heather',
  'Middle_Name__c': 'Nicole',
  'Birth_Date__c': datetime.date(1994, 6, 7)}]

# Insert Staff_data to Salesforce

In [181]:
for rec in staff_data_records:
    record = {
        'EmployeeID__c': rec['EmployeeID__c'],
        'Last_Name__c': rec['Last_Name__c'],
        'First_Name__c': rec['First_Name__c'],
        'Middle_Name__c': rec['Middle_Name__c'],
        #'Birth_Date__c': rec['Birth_Date__c'],
    }
    try:
        sf.Staff__c.create(record)
    except Exception as e:
        print(e)

## Create Staff Lookup TAble

In [182]:
sf_staff_records = []

sf_staff_data = sf.query('SELECT Name, EmployeeID__c FROM Staff__c')

for row in sf_staff_data['records']:
    
    rec = {
        'ID_Staff__c': row['Name'],
        'Staff_ID__c': row['EmployeeID__c']
    }
    
    sf_staff_records.append(rec)
    
sf_staff_xwalk = pd.DataFrame(sf_staff_records)
sf_staff_xwalk.head()

,ID_Staff__c,Staff_ID__c
0,a084x000000dnu0,000160655
1,a084x000000dnu5,000159108
2,a084x000000dntq,000184220
3,a084x000000dntv,000130109


In [183]:
query = '''

    SELECT
        s.EmployeeID AS Staff_ID__c,
        co.CourseCode AS Course_Code__c,
        cl.Section AS Section__c,
        sa.Role AS Role__c,
        sa.StartDate AS Start_Date__c,
        sa.EndDate AS End_Date__c
    FROM 
        staffassignment sa
        INNER JOIN class cl
        ON sa.ID_Class = cl.ID_Class
        INNER JOIN course co
        ON cl.ID_Course = co.ID_Course
        INNER JOIN staff s
        ON sa.ID_Staff = s.ID_Staff

'''

staffassignment_df = pd.read_sql(query, conn)
staffassignment_df.head()

,Staff_ID__c,Course_Code__c,Section__c,Role__c,Start_Date__c,End_Date__c
0,000184220,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,Teacher Assistant,2020-09-16,None
1,000159108,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,Teacher Assistant,2020-09-16,None
2,000160655,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,Instructor,2020-09-16,None


In [195]:
sf_staff_xwalk
#sf_student_xwalk

staff_assignment_load = pd.merge(staffassignment_df, sf_staff_xwalk, on=['Staff_ID__c'])

staff_assignment_load = pd.merge(staff_assignment_load, sf_staff_xwalk)

staff_assignment_load = pd.merge(staff_assignment_load, sf_class_xwalk)


staff_assignment_load = staff_assignment_load[['ID_Staff__c','ID_Class__c','Role__c','Start_Date__c','End_Date__c']]

staff_assignment_load['Start_Date__c'] = staff_assignment_load['Start_Date__c'].astype(str)

staff_assignment_load

staff_assignment_records = staff_assignment_load.to_dict(orient='records')
staff_assignment_records

[{'ID_Staff__c': 'a084x000000dntq',
  'ID_Class__c': 'a014x00000A1DjW',
  'Role__c': 'Teacher Assistant',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Staff__c': 'a084x000000dnu5',
  'ID_Class__c': 'a014x00000A1DjW',
  'Role__c': 'Teacher Assistant',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Staff__c': 'a084x000000dnu0',
  'ID_Class__c': 'a014x00000A1DjW',
  'Role__c': 'Instructor',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None}]

# Insert Staffassigment Records into Salesforce

In [196]:
for row in staff_assignment_records:
    try:
        sf.Staff_Assignment__c.create(row)
    except Exception as e:
        print(e)

# Prepare ETL for the Student data

In [55]:
student_data_df = pd.read_sql("SELECT * FROM student", conn)
student_data_df.head(5)

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Dartling,Heather,Alice,None,F
1,34,25003514,Dartling,Lana,Cecille,None,F
2,35,25005833,Dartling,Jessica,Dorothy,None,F
3,36,25002589,Dartling,Kimberly,Genevieve,None,F
4,37,25007185,Dartling,Katherine,Lynnette,None,F


In [56]:
student_data_df.rename(columns={
    'StudentID':'StudentID__c',
    'LastName':'Last_Name__c',
    'FirstName':'First_Name__c',
    'MiddleName':'Middle_Name__c',
    'BirthDate':'Birth_Date__c',
    'Gender':'Gender__c'
}, inplace=True)
student_data_df.head()

,ID_Student,StudentID__c,Last_Name__c,First_Name__c,Middle_Name__c,Birth_Date__c,Gender__c
0,33,25004961,Dartling,Heather,Alice,None,F
1,34,25003514,Dartling,Lana,Cecille,None,F
2,35,25005833,Dartling,Jessica,Dorothy,None,F
3,36,25002589,Dartling,Kimberly,Genevieve,None,F
4,37,25007185,Dartling,Katherine,Lynnette,None,F


In [57]:
student_data_df = student_data_df[['StudentID__c', 'Last_Name__c','First_Name__c', 'Middle_Name__c','Birth_Date__c','Gender__c']]
student_data_df.head()

,StudentID__c,Last_Name__c,First_Name__c,Middle_Name__c,Birth_Date__c,Gender__c
0,25004961,Dartling,Heather,Alice,None,F
1,25003514,Dartling,Lana,Cecille,None,F
2,25005833,Dartling,Jessica,Dorothy,None,F
3,25002589,Dartling,Kimberly,Genevieve,None,F
4,25007185,Dartling,Katherine,Lynnette,None,F


In [58]:
student_data_records = student_data_df.to_dict('records')
student_data_records

[{'StudentID__c': '25004961',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Heather',
  'Middle_Name__c': 'Alice',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25003514',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Lana',
  'Middle_Name__c': 'Cecille',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25005833',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Jessica',
  'Middle_Name__c': 'Dorothy',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25002589',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Kimberly',
  'Middle_Name__c': 'Genevieve',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25007185',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Katherine',
  'Middle_Name__c': 'Lynnette',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25006014',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Precious',
  'Middle_Name__c': 'Mariette',
  'Birth_Date__c': None,
  'Gender__c': '

# Insert Student Records into Salesforce

In [59]:
for rec in student_data_records:

    record = {
        'StudentID__c': rec['StudentID__c'],
        'Last_Name__c': rec['Last_Name__c'],
        'First_Name__c': rec['First_Name__c'],
        'Middle_Name__c': rec['Middle_Name__c'],
        'Birth_Date__c': rec['Birth_Date__c'],
        'Gender__c': rec['Gender__c']
    }
    
    try:
        sf.Student__c.create(record)
    except Exception as e:
        print(e)

# Create Student Lookup Table

In [96]:
student_lookup_list = []
# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT StudentID__c, Name FROM Student__c")
for row in data:
    rec = {
        'ID_Student__c': row['Name'], # this is a critical line of code
        'StudentID__c': row['StudentID__c']
    }
    student_lookup_list.append(rec)
student_lookup_list

[{'ID_Student__c': 'a064x000004SgeS', 'StudentID__c': '25002876'},
 {'ID_Student__c': 'a064x000004Sgcv', 'StudentID__c': '25003605'},
 {'ID_Student__c': 'a064x000004Sgde', 'StudentID__c': '25005284'},
 {'ID_Student__c': 'a064x000004Sgdt', 'StudentID__c': '25005250'},
 {'ID_Student__c': 'a064x000004Sgdo', 'StudentID__c': '25002555'},
 {'ID_Student__c': 'a064x000004SgdZ', 'StudentID__c': '25002714'},
 {'ID_Student__c': 'a064x000004Sgec', 'StudentID__c': '25007334'},
 {'ID_Student__c': 'a064x000004SgeD', 'StudentID__c': '25005443'},
 {'ID_Student__c': 'a064x000004Sge3', 'StudentID__c': '25003773'},
 {'ID_Student__c': 'a064x000004SgdF', 'StudentID__c': '25002311'},
 {'ID_Student__c': 'a064x000004Sgd5', 'StudentID__c': '25004152'},
 {'ID_Student__c': 'a064x000004Sgd0', 'StudentID__c': '25002056'},
 {'ID_Student__c': 'a064x000004SgdA', 'StudentID__c': '25004097'},
 {'ID_Student__c': 'a064x000004Sgcb', 'StudentID__c': '25007185'},
 {'ID_Student__c': 'a064x000004SgeI', 'StudentID__c': '2500676

In [97]:
student_lookup_df = pd.DataFrame(student_lookup_list)
student_lookup_df

,ID_Student__c,StudentID__c
0,a064x000004SgeS,25002876
1,a064x000004Sgcv,25003605
2,a064x000004Sgde,25005284
3,a064x000004Sgdt,25005250
4,a064x000004Sgdo,25002555
5,a064x000004SgdZ,25002714
6,a064x000004Sgec,25007334
7,a064x000004SgeD,25005443
8,a064x000004Sge3,25003773
9,a064x000004SgdF,25002311


In [186]:
sf_course_records = []

sf_course_data = sf.query('SELECT Name, Course_Code__c FROM Course__c')

for row in sf_course_data['records']:
    
    rec = {
        'ID_Course__c': row['Name'],
        'Course_Code__c': row['Course_Code__c']
    }
    
    sf_course_records.append(rec)
    
sf_course_df = pd.DataFrame(sf_course_records)
sf_course_df.head()

,ID_Course__c,Course_Code__c
0,a004x000003VU7c,BC-UIUX
1,a004x000003VU7r,CIS-438
2,a004x000003VU7m,CIS-405
3,a004x000003VU7h,CIS-349
4,a004x000003VU7S,BC-DATAVIZ


In [187]:
sf_class_records = []

sf_class_data = sf.query('SELECT ID_Course__c, Name, Section__c FROM Class__c')

for row in sf_class_data['records']:
    
    rec = {
        'ID_Course__c': row['ID_Course__c'],
        'ID_Class__c': row['Name'],
        'Section__c': row['Section__c']
    }
    
    sf_class_records.append(rec)
    
# temporary workaround to strip last three characters from ID_Course__c
for rec in sf_class_records:
    rec['ID_Course__c'] = rec['ID_Course__c'][:-3]
    
sf_class_df = pd.DataFrame(sf_class_records)
sf_class_df.head()

,ID_Course__c,ID_Class__c,Section__c
0,a004x000003VU7c,a014x00000A1Djl,GWARL201905UIUX3
1,a004x000003VU7S,a014x00000A1DjW,GWU-ARL-DATA-PT-09-0
2,a004x000003VU7X,a014x00000A1Djg,GWARL201905WEB3
3,a004x000003VU7S,a014x00000A1Djb,GWDC201805DATA3


In [188]:
sf_class_xwalk = pd.merge(sf_class_df, sf_course_df)
sf_class_xwalk

,ID_Course__c,ID_Class__c,Section__c,Course_Code__c
0,a004x000003VU7c,a014x00000A1Djl,GWARL201905UIUX3,BC-UIUX
1,a004x000003VU7S,a014x00000A1DjW,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
2,a004x000003VU7S,a014x00000A1Djb,GWDC201805DATA3,BC-DATAVIZ
3,a004x000003VU7X,a014x00000A1Djg,GWARL201905WEB3,BC-WEBDEV


In [189]:
sf_student_records = []

sf_student_data = sf.query('SELECT Name, StudentID__c FROM Student__c')

for row in sf_student_data['records']:
    
    rec = {
        'ID_Student__c': row['Name'],
        'Student_ID__c': row['StudentID__c']
    }
    
    sf_student_records.append(rec)
    
sf_student_xwalk = pd.DataFrame(sf_student_records)
sf_student_xwalk.head()

,ID_Student__c,Student_ID__c
0,a064x000004SgeS,25002876
1,a064x000004Sgcv,25003605
2,a064x000004Sgde,25005284
3,a064x000004Sgdt,25005250
4,a064x000004Sgdo,25002555


In [190]:
query = '''

    SELECT
        s.StudentID AS Student_ID__c,
        co.CourseCode AS Course_Code__c,
        cl.Section AS Section__c,
        cp.StartDate AS Start_Date__c,
        cp.EndDate AS End_Date__c
    FROM 
        classparticipant cp
        INNER JOIN class cl
        ON cp.ID_Class = cl.ID_Class
        INNER JOIN course co
        ON cl.ID_Course = co.ID_Course
        INNER JOIN student s
        ON cp.ID_Student = s.ID_Student

'''

classparticiant_df = pd.read_sql(query, conn)
classparticiant_df.head()

,Student_ID__c,Course_Code__c,Section__c,Start_Date__c,End_Date__c
0,25004961,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
1,25003514,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
2,25005833,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
3,25007334,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
4,25002589,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None


In [191]:
print(classparticiant_df.columns)
print(sf_class_xwalk.columns)

Index(['Student_ID__c', 'Course_Code__c', 'Section__c', 'Start_Date__c',
       'End_Date__c'],
      dtype='object')
Index(['ID_Course__c', 'ID_Class__c', 'Section__c', 'Course_Code__c'], dtype='object')


In [192]:
sf_class_xwalk
#sf_student_xwalk

class_participant_load = pd.merge(classparticiant_df, sf_class_xwalk, on=['Course_Code__c', 'Section__c'])

class_participant_load = pd.merge(class_participant_load, sf_student_xwalk)

class_participant_load = class_participant_load[['ID_Student__c','ID_Class__c','Start_Date__c','End_Date__c']]

class_participant_load['Start_Date__c'] = class_participant_load['Start_Date__c'].astype(str)

class_participant_load

class_participant_records = class_participant_load.to_dict(orient='records')
class_participant_records

[{'ID_Student__c': 'a064x000004SgcH',
  'ID_Class__c': 'a014x00000A1DjW',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a064x000004SgcM',
  'ID_Class__c': 'a014x00000A1DjW',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a064x000004SgcR',
  'ID_Class__c': 'a014x00000A1DjW',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a064x000004Sgec',
  'ID_Class__c': 'a014x00000A1DjW',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a064x000004SgcW',
  'ID_Class__c': 'a014x00000A1DjW',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a064x000004Sgcb',
  'ID_Class__c': 'a014x00000A1DjW',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a064x000004Sgcg',
  'ID_Class__c': 'a014x00000A1DjW',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a064x000004Sgcl',
  'ID_Class__c': 'a014x00000A1DjW',
  'Start_

# Insert Classparticipant Records into Salesforce

In [193]:
for row in class_participant_records:
    try:
        sf.Class_Participant__c.create(row)
    except Exception as e:
        print(e)

## Example of Deleting Records

Select the IDs of the records first and then process the results.

Ultimately, you want a list of IDs in the end.


In [20]:
class_records = sf.query("SELECT Id FROM Course__c")
recs_to_delete = [{'Id': r['Id']} for r in class_records['records']]
recs_to_delete

[{'Id': 'a004x000003VIHEAA4'},
 {'Id': 'a004x000003VIHJAA4'},
 {'Id': 'a004x000003VIHOAA4'},
 {'Id': 'a004x000003VIHTAA4'},
 {'Id': 'a004x000003VIHYAA4'},
 {'Id': 'a004x000003VIHdAAO'},
 {'Id': 'a004x000003VIHiAAO'},
 {'Id': 'a004x000003VIHnAAO'}]

In [21]:
#sf.bulk.Course__c.delete(recs_to_delete)

In [22]:
#or rec in recs_to_delete:
    try:
        sf.Course__c.delete(rec['Id'])
    except Exception as e:
        print(e)

IndentationError: unexpected indent (<ipython-input-22-177b1daf3026>, line 2)

In [152]:
sa_records = sf.query("SELECT Name FROM Staff_Assignment__c")
recs_to_delete = [{'Id': r['Name']} for r in sa_records['records']]
sf.bulk.Staff_Assignment__c.delete(recs_to_delete)

[{'success': True, 'created': False, 'id': 'a094x000000ahdYAAQ', 'errors': []},
 {'success': True, 'created': False, 'id': 'a094x000000ahebAAA', 'errors': []},
 {'success': True, 'created': False, 'id': 'a094x000000aheHAAQ', 'errors': []},
 {'success': True, 'created': False, 'id': 'a094x000000ahdEAAQ', 'errors': []},
 {'success': True, 'created': False, 'id': 'a094x000000ahdTAAQ', 'errors': []},
 {'success': True, 'created': False, 'id': 'a094x000000ahe2AAA', 'errors': []},
 {'success': True, 'created': False, 'id': 'a094x000000ahegAAA', 'errors': []},
 {'success': True, 'created': False, 'id': 'a094x000000ahelAAA', 'errors': []},
 {'success': True, 'created': False, 'id': 'a094x000000aheqAAA', 'errors': []},
 {'success': True, 'created': False, 'id': 'a094x000000ahdOAAQ', 'errors': []},
 {'success': True, 'created': False, 'id': 'a094x000000ahdnAAA', 'errors': []},
 {'success': True, 'created': False, 'id': 'a094x000000aheRAAQ', 'errors': []},
 {'success': True, 'created': False, 'id

In [153]:
cp_records = sf.query("SELECT Name FROM Class_Participant__c")
recs_to_delete = [{'Id': r['Name']} for r in cp_records['records']]
sf.bulk.Class_Participant__c.delete(recs_to_delete)

[{'success': True, 'created': False, 'id': 'a074x000001GeorAAC', 'errors': []},
 {'success': True, 'created': False, 'id': 'a074x000001GeosAAC', 'errors': []},
 {'success': True, 'created': False, 'id': 'a074x000001GenUAAS', 'errors': []},
 {'success': True, 'created': False, 'id': 'a074x000001GeqdAAC', 'errors': []},
 {'success': True, 'created': False, 'id': 'a074x000001GepaAAC', 'errors': []},
 {'success': True, 'created': False, 'id': 'a074x000001GergAAC', 'errors': []},
 {'success': True, 'created': False, 'id': 'a074x000001GepbAAC', 'errors': []},
 {'success': True, 'created': False, 'id': 'a074x000001GenjAAC', 'errors': []},
 {'success': True, 'created': False, 'id': 'a074x000001Gem7AAC', 'errors': []},
 {'success': True, 'created': False, 'id': 'a074x000001GepfAAC', 'errors': []},
 {'success': True, 'created': False, 'id': 'a074x000001Get8AAC', 'errors': []},
 {'success': True, 'created': False, 'id': 'a074x000001GerqAAC', 'errors': []},
 {'success': True, 'created': False, 'id

In [154]:
class_records = sf.query("SELECT Name FROM Class__c")
recs_to_delete = [{'Id': r['Name']} for r in class_records['records']]
sf.bulk.Class__c.delete(recs_to_delete)

[{'success': True, 'created': False, 'id': 'a014x000008WKpSAAW', 'errors': []},
 {'success': True, 'created': False, 'id': 'a014x000008W0ejAAC', 'errors': []},
 {'success': True, 'created': False, 'id': 'a014x000008W0eeAAC', 'errors': []},
 {'success': True, 'created': False, 'id': 'a014x000008W0eZAAS', 'errors': []},
 {'success': True, 'created': False, 'id': 'a014x000008W0eoAAC', 'errors': []},
 {'success': True, 'created': False, 'id': 'a014x000008WKpXAAW', 'errors': []},
 {'success': True, 'created': False, 'id': 'a014x000008WKpmAAG', 'errors': []},
 {'success': True, 'created': False, 'id': 'a014x000008WKprAAG', 'errors': []},
 {'success': True, 'created': False, 'id': 'a014x000008WKpIAAW', 'errors': []},
 {'success': True, 'created': False, 'id': 'a014x000008WKpNAAW', 'errors': []},
 {'success': True, 'created': False, 'id': 'a014x000008WKpcAAG', 'errors': []},
 {'success': True, 'created': False, 'id': 'a014x000008WJAXAA4', 'errors': []},
 {'success': True, 'created': False, 'id

In [155]:
staff_records = sf.query("SELECT Name FROM Staff__c")
recs_to_delete = [{'Id': r['Name']} for r in staff_records['records']]
sf.bulk.Staff__c.delete(recs_to_delete)

[{'success': True, 'created': False, 'id': 'a084x000000dnTcAAI', 'errors': []},
 {'success': True, 'created': False, 'id': 'a084x000000dnThAAI', 'errors': []},
 {'success': True, 'created': False, 'id': 'a084x000000dnTrAAI', 'errors': []},
 {'success': True, 'created': False, 'id': 'a084x000000dnJ9AAI', 'errors': []},
 {'success': True, 'created': False, 'id': 'a084x000000dnJ4AAI', 'errors': []},
 {'success': True, 'created': False, 'id': 'a084x000000dnIuAAI', 'errors': []},
 {'success': True, 'created': False, 'id': 'a084x000000dnIzAAI', 'errors': []},
 {'success': True, 'created': False, 'id': 'a084x000000dnTmAAI', 'errors': []}]

In [158]:
student_records = sf.query("SELECT Name FROM Student__c")
recs_to_delete = [{'Id': r['Name']} for r in class_records['records']]
sf.bulk.Student__c.delete(recs_to_delete)

[{'success': False,
  'created': False,
  'id': None,
  'errors': [{'message': "Invalid Id for entity type 'Student__c'",
    'fields': [],
    'statusCode': 'INVALID_ID_FIELD',
    'extendedErrorDetails': None}]},
 {'success': False,
  'created': False,
  'id': None,
  'errors': [{'message': "Invalid Id for entity type 'Student__c'",
    'fields': [],
    'statusCode': 'INVALID_ID_FIELD',
    'extendedErrorDetails': None}]},
 {'success': False,
  'created': False,
  'id': None,
  'errors': [{'message': "Invalid Id for entity type 'Student__c'",
    'fields': [],
    'statusCode': 'INVALID_ID_FIELD',
    'extendedErrorDetails': None}]},
 {'success': False,
  'created': False,
  'id': None,
  'errors': [{'message': "Invalid Id for entity type 'Student__c'",
    'fields': [],
    'statusCode': 'INVALID_ID_FIELD',
    'extendedErrorDetails': None}]},
 {'success': False,
  'created': False,
  'id': None,
  'errors': [{'message': "Invalid Id for entity type 'Student__c'",
    'fields': [],


In [157]:
course_records = sf.query("SELECT Name FROM Course__c")
recs_to_delete = [{'Id': r['Name']} for r in course_records['records']]
sf.bulk.Course__c.delete(recs_to_delete)

[{'success': True, 'created': False, 'id': 'a004x000003VIHJAA4', 'errors': []},
 {'success': True, 'created': False, 'id': 'a004x000003VIHYAA4', 'errors': []},
 {'success': True, 'created': False, 'id': 'a004x000003VIHEAA4', 'errors': []},
 {'success': True, 'created': False, 'id': 'a004x000003VIHiAAO', 'errors': []},
 {'success': True, 'created': False, 'id': 'a004x000003VIHTAA4', 'errors': []},
 {'success': True, 'created': False, 'id': 'a004x000003VQ6DAAW', 'errors': []},
 {'success': True, 'created': False, 'id': 'a004x000003VQ6NAAW', 'errors': []},
 {'success': True, 'created': False, 'id': 'a004x000003VQ5yAAG', 'errors': []},
 {'success': True, 'created': False, 'id': 'a004x000003VIHdAAO', 'errors': []},
 {'success': True, 'created': False, 'id': 'a004x000003VIHOAA4', 'errors': []},
 {'success': True, 'created': False, 'id': 'a004x000003VIHnAAO', 'errors': []},
 {'success': True, 'created': False, 'id': 'a004x000003VQ63AAG', 'errors': []},
 {'success': True, 'created': False, 'id